In [2]:
# 機械学習のライブラリ関連をインポート
import pandas as pd
import numpy as np

# ランダムにシャッフルして，学習・テストに分割するモジュール
from sklearn.model_selection import ShuffleSplit

# 深層学習のライブラリをインポート
import tensorflow as tf
import tensorflow.keras as keras

#表示系のインポートと設定
%matplotlib inline
import matplotlib.pyplot as plt

# ファイル操作
import os

# 画像操作
from PIL import Image

In [66]:
# DIRS
DIRS_DATASET = "../Training/Datasets10000/"

In [67]:
file_names = []
    
# フォルダ内のファイルを取得
files = os.listdir(DIRS_DATASET)

# ファイル名を配列に格納
for file in files:
    file_names.append(file)

# Pythonリスト型をnumpy.ndarray型に変換
file_names = np.array(file_names)


In [68]:
file_names

array(['4510033397528.jpeg', '4510050634699.jpeg', '4510072948944.jpeg',
       ..., '4999988925606.jpeg', '4999998682512.jpeg',
       '4999999421189.jpeg'], dtype='<U18')

In [69]:
# # 学習データとテストデータのインデックスを作成
# train_index, test_index = next(ss.split(file_names))

In [70]:
# file_names_train, file_names_test = file_names[train_index], file_names[test_index] # 学習データ，テストデータ

### 画像をnumpy配列にするための関数

In [71]:
def jpeg_to_numpy(image_path):
    # JPEG画像を開く
    image = Image.open(image_path)
    # NumPy配列に変換
    numpy_array = np.array(image)
    
    return numpy_array

def convert_to_grayscale(numpy_array):
    # グレーと言わず2値化
    grayscale_array = np.where(numpy_array <= 128, 0, 255)
    # plt.imshow(grayscale_array) # こいつらのせいで処理が重かった。出力系は要注意
    # print(grayscale_array)
    return grayscale_array


### 画像を配列にしてよしなに

In [72]:
X, y = [], []

In [73]:
for file_name in file_names:
    numpy_array = jpeg_to_numpy(DIRS_DATASET + file_name)
    grayscale_array = convert_to_grayscale(numpy_array)
    for i in range(13):
        X.append(np.where(grayscale_array == 255, i+1, 0))
        y.append(file_name[i])

In [74]:
X = np.array(X)
y = np.array(y, dtype=int)

In [75]:
X

array([[[[ 0,  0,  0],
         [ 1,  1,  1],
         [ 0,  0,  0],
         ...,
         [ 1,  1,  1],
         [ 1,  1,  1],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 2,  2,  2],
         [ 0,  0,  0],
         ...,
         [ 2,  2,  2],
         [ 2,  2,  2],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 3,  3,  3],
         [ 0,  0,  0],
         ...,
         [ 3,  3,  3],
         [ 3,  3,  3],
         [ 0,  0,  0]]],


       ...,


       [[[ 0,  0,  0],
         [11, 11, 11],
         [ 0,  0,  0],
         ...,
         [11, 11, 11],
         [11, 11, 11],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [12, 12, 12],
         [ 0,  0,  0],
         ...,
         [12, 12, 12],
         [12, 12, 12],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [13, 13, 13],
         [ 0,  0,  0],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [ 0,  0,  0]]]])

In [76]:
y

array([4, 5, 1, ..., 1, 8, 9])

In [77]:
# ラベルデータをone-hotベクトルに直す
labels = {
    0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
    2: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 
    3: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 
    4: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 
    5: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 
    6: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 
    7: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 
    8: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 
    9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
}

y = np.array(list(map(lambda v : labels[v] , y)))

In [78]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

### データの分割

In [79]:
ss = ShuffleSplit(n_splits=1,      # 分割を1個生成
                  train_size=0.8,  # 学習
                  test_size =0.2,  # テスト
                  random_state=0)  # 乱数種（再現用）

# 学習データとテストデータのインデックスを作成
train_index, test_index = next(ss.split(X))

X_train, X_test = X[train_index], X[test_index] # 学習データ，テストデータ
y_train, y_test = y[train_index], y[test_index] # 学習データのラベル，テストデータのラベル

In [80]:
X

array([[[[ 0,  0,  0],
         [ 1,  1,  1],
         [ 0,  0,  0],
         ...,
         [ 1,  1,  1],
         [ 1,  1,  1],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 2,  2,  2],
         [ 0,  0,  0],
         ...,
         [ 2,  2,  2],
         [ 2,  2,  2],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 3,  3,  3],
         [ 0,  0,  0],
         ...,
         [ 3,  3,  3],
         [ 3,  3,  3],
         [ 0,  0,  0]]],


       ...,


       [[[ 0,  0,  0],
         [11, 11, 11],
         [ 0,  0,  0],
         ...,
         [11, 11, 11],
         [11, 11, 11],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [12, 12, 12],
         [ 0,  0,  0],
         ...,
         [12, 12, 12],
         [12, 12, 12],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [13, 13, 13],
         [ 0,  0,  0],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [ 0,  0,  0]]]])

In [81]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [82]:
X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3] 

(104000, 1, 112, 3)

In [83]:
X_train

array([[[[ 0,  0,  0],
         [ 3,  3,  3],
         [ 0,  0,  0],
         ...,
         [ 3,  3,  3],
         [ 3,  3,  3],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 1,  1,  1],
         [ 0,  0,  0],
         ...,
         [ 1,  1,  1],
         [ 1,  1,  1],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 2,  2,  2],
         [ 0,  0,  0],
         ...,
         [ 2,  2,  2],
         [ 2,  2,  2],
         [ 0,  0,  0]]],


       ...,


       [[[ 0,  0,  0],
         [13, 13, 13],
         [ 0,  0,  0],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 5,  5,  5],
         [ 0,  0,  0],
         ...,
         [ 5,  5,  5],
         [ 5,  5,  5],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 6,  6,  6],
         [ 0,  0,  0],
         ...,
         [ 6,  6,  6],
         [ 6,  6,  6],
         [ 0,  0,  0]]]])

### ニューラルネットワークの構築

In [84]:

# 学習し、テストデータで評価し、スコアを表示する
# 引数は、中間層の数、バッチサイズ、epoch数

def fit_epoch(neuron, batch, epochs):
    
    # レイヤーのオブジェクトを作成
    Dense = keras.layers.Dense

    # モデルの構造を定義
    model = keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(1, 112, 3)))
    model.add(Dense(neuron, activation='relu'))
    
    # 畳み込み層を追加
    # model.add(tf.keras.layers.Flatten(tf.keras.layers.Conv2D(filters=neuron, kernel_size=(3, 3), activation='relu', input_shape=(1, 112, 3))))

    model.add(Dense(10, activation='softmax')) # 10つのラベルがありsoftmaxで最後の層作る

    # モデルを構築
    model.compile(
    loss='categorical_crossentropy',
    optimizer='Adamax',
    metrics=['accuracy'])

    # 学習を実行
    hist = model.fit(X_train, y_train,
        batch_size=batch, # 誤差逆伝播法をするときの1回当たりのデータ数
        epochs=epochs,
        verbose=1,
        validation_split=0.1)
    
   # モデルを評価
    score = model.evaluate(X_test, y_test, verbose=1)
    print('正解率=', score[1], 'loss=', score[0])
    
     # 学習の様子をグラフへ描画 
    # 正解率の推移をプロット
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('Accuracy')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # ロスの推移をプロット
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Loss')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()   

In [88]:
%%time
fit_epoch(128, 8, 2000)

Epoch 1/2000
11700/11700 [==============================] - 23s 2ms/step - loss: 2.2183 - accuracy: 0.2321 - val_loss: 2.0911 - val_accuracy: 0.2493
Epoch 2/2000
11700/11700 [==============================] - 20s 2ms/step - loss: 2.0573 - accuracy: 0.2473 - val_loss: 2.0029 - val_accuracy: 0.2527
Epoch 3/2000
11700/11700 [==============================] - 20s 2ms/step - loss: 1.9308 - accuracy: 0.2901 - val_loss: 1.8807 - val_accuracy: 0.3310
Epoch 4/2000
11700/11700 [==============================] - 20s 2ms/step - loss: 1.8477 - accuracy: 0.3338 - val_loss: 1.8333 - val_accuracy: 0.3402
Epoch 5/2000
11700/11700 [==============================] - 20s 2ms/step - loss: 1.8127 - accuracy: 0.3375 - val_loss: 1.8113 - val_accuracy: 0.3369
Epoch 6/2000
11700/11700 [==============================] - 20s 2ms/step - loss: 1.7942 - accuracy: 0.3404 - val_loss: 1.7968 - val_accuracy: 0.3362
Epoch 7/2000
11700/11700 [==============================] - 20s 2ms/step - loss: 1.7824 - accuracy: 0.3440

KeyboardInterrupt: 